In [1]:
import pandas as pd
import datetime

#Load the game data to pandas data frame
df_season_map  = pd.read_csv('../data/game_dates_season.csv')
df_raw = pd.read_csv('../data/nfl_games_raw.csv')
df_teams = pd.read_csv('../data/teams.csv')

In [2]:
df_games = df_raw[['Date','Home Team','Away Team','Home Score','Away Score','Playoff Game?', 'Home Line Close','Away Line Close', 'Total Score Close']]
df_games = pd.merge(df_games, df_season_map, on='Date', how='left')
df_games = df_games.drop(columns=['Unnamed: 0'])

df_teams_abbr = df_teams[['Team Name','Team Abbr']]
df_games = pd.merge(df_games, df_teams_abbr, left_on='Home Team', right_on='Team Name', how='left')
df_games = df_games.rename(columns={'Team Abbr':'Home Team Abbr'})
df_games = df_games.drop(columns=['Team Name'])
df_games = pd.merge(df_games, df_teams_abbr, left_on='Away Team', right_on='Team Name', how='left')
df_games = df_games.rename(columns={'Team Abbr':'Away Team Abbr'})
df_games = df_games.drop(columns=['Team Name'])
df_games = df_games[df_games['Playoff Game?'].isna()]
df_games = df_games.drop(columns=['Playoff Game?'])
print(df_games.head())

         Date           Home Team              Away Team  Home Score  \
0  2024-09-23  Cincinnati Bengals  Washington Commanders          33   
1  2024-09-23       Buffalo Bills   Jacksonville Jaguars          47   
2  2024-09-22     Atlanta Falcons     Kansas City Chiefs          17   
3  2024-09-22   Arizona Cardinals          Detroit Lions          13   
4  2024-09-22      Dallas Cowboys       Baltimore Ravens          25   

   Away Score  Home Line Close  Away Line Close  Total Score Close  season  \
0          38             -7.5              7.5               46.0    2024   
1          10             -4.0              4.0               46.5    2024   
2          22              3.5             -3.5               46.5    2024   
3          20              3.0             -3.0               51.5    2024   
4          28              1.5             -1.5               47.5    2024   

   week Home Team Abbr Away Team Abbr  
0     3            CIN            WAS  
1     3           

In [3]:
df_games['winner'] = df_games.apply(lambda row: row['Home Team Abbr'] if row['Home Score'] > row['Away Score'] else row['Away Team Abbr'], axis=1)
df_games['loser'] = df_games.apply(lambda row: row['Away Team Abbr'] if row['Home Score'] > row['Away Score'] else row['Home Team Abbr'], axis=1)
df_games['margin_victory'] = abs(df_games['Home Score'] - df_games['Away Score'])
df_games['total_score'] = df_games['Home Score'] + df_games['Away Score']
df_games['margin_perc'] = df_games['margin_victory'] / df_games['total_score']
df_games['home_team_winner'] = df_games['Home Score'] > df_games['Away Score']

print(df_games.head())

         Date           Home Team              Away Team  Home Score  \
0  2024-09-23  Cincinnati Bengals  Washington Commanders          33   
1  2024-09-23       Buffalo Bills   Jacksonville Jaguars          47   
2  2024-09-22     Atlanta Falcons     Kansas City Chiefs          17   
3  2024-09-22   Arizona Cardinals          Detroit Lions          13   
4  2024-09-22      Dallas Cowboys       Baltimore Ravens          25   

   Away Score  Home Line Close  Away Line Close  Total Score Close  season  \
0          38             -7.5              7.5               46.0    2024   
1          10             -4.0              4.0               46.5    2024   
2          22              3.5             -3.5               46.5    2024   
3          20              3.0             -3.0               51.5    2024   
4          28              1.5             -1.5               47.5    2024   

   week Home Team Abbr Away Team Abbr winner loser  margin_victory  \
0     3            CIN      

In [21]:
df_games.to_csv('../data/nfl_games.csv')